# Preprocessing Part 1 - Partitioning and Data Mining Transaction Data in transactions_train.csv

### Run on ml.t3.xlarge instance

In [1]:
%%capture
!pip install numpy
!pip install pandas
!pip install boto3
!pip install matplotlib
!pip install seaborn
!pip install datetime
!pip install awswrangler

In [2]:
import numpy as np
import pandas as pd
import boto3
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import awswrangler as wr

pd.options.display.float_format = '{:.4f}'.format
pd.options.mode.chained_assignment = None

### Loading Data via S3

In [3]:
bucket='ads-508-group-6-raw'
data_key = 'transactions_train.csv'

s3 = boto3.client('s3')
transactions_obj = s3.get_object(Bucket = bucket, Key = data_key)

transactions_df = pd.read_csv(transactions_obj['Body'])

### Loading Data Locally

In [4]:
# transactions_df = pd.read_csv("..\\data\\transactions_train.csv")

In [5]:
transactions_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.0508,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.0305,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.0152,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.0169,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.0169,2


### Recast Variables

In [6]:
transactions_df['t_dat'] = pd.to_datetime(transactions_df['t_dat'], format='%Y-%m-%d')
transactions_df = transactions_df.astype({'article_id':'str',
                                          'sales_channel_id':'str'})

## Partitioning transaction_train.csv

Since we are trying to predict whether or not a customer will churn, the dataset needs to be split to prevent data leakage in our predictor variables. For the purposes of defining churn, we will be attempting to predict whether Q1 customers will churn in Q2 of 2020.

In [7]:
target_df = transactions_df.loc[(transactions_df['t_dat'] >= '2020-04-01') & (transactions_df['t_dat'] <= '2020-06-30')]

In [8]:
current_df = transactions_df.loc[(transactions_df['t_dat'] >= '2020-01-01') & (transactions_df['t_dat'] <= '2020-03-31')]

In [9]:
historical_df = transactions_df.loc[(transactions_df['t_dat'] <= '2020-03-31')]

## Generating Target Labels

In [10]:
target_df = target_df.drop_duplicates(subset = ['customer_id'])

In [11]:
target_df['churn'] = 0

In [12]:
target_df = target_df.drop(columns=['t_dat', 
                                    'article_id', 
                                    'price', 
                                    'sales_channel_id'])

In [13]:
target_df.head()

,customer_id,churn
23934157,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0
23934159,000563485cbb7850b0a93c6606f89c5b961c6647d1bd48...,0
23934161,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,0
23934162,000eae69313b4fc1824fa7e439f168cc140bf4c3f3a7e9...,0
23934165,000ec422ba5459f0295c1e86872d61b746bb8f84345efd...,0


## Data Mining Historical Transaction Data

In [14]:
historical_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.0508,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.0305,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.0152,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.0169,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.0169,2


In [15]:
historical_df['t_dat_max'] = historical_df['t_dat'].max()
historical_df['customer_t_dat_max'] = historical_df['t_dat']

In [16]:
def max_min(x):
    return x.max() - x.min()

In [17]:
historical_df = historical_df.groupby('customer_id').agg({'t_dat_max':'max', 
                                                          'customer_t_dat_max': 'max',
                                                          't_dat':max_min, 
                                                          'article_id':'count', 
                                                          'price':'sum'})

In [18]:
historical_df = historical_df.reset_index()

In [19]:
historical_df['t_dat'] = pd.to_numeric(historical_df['t_dat'].dt.days, downcast='integer')

In [20]:
historical_df['days_since_last_purchase'] = historical_df['t_dat_max'] - historical_df['customer_t_dat_max']
historical_df['lifetime_average_amount_spent_per_article'] = historical_df['price']/historical_df['article_id']
historical_df['lifetime_average_days_between_purchase'] = historical_df['article_id']/historical_df['t_dat']

In [21]:
historical_df['lifetime_average_days_between_purchase'].replace(np.inf, 0, inplace = True)

In [22]:
historical_df['days_since_last_purchase'] = pd.to_numeric(historical_df['days_since_last_purchase'].dt.days, downcast='integer')

In [23]:
historical_df = historical_df.rename(columns={"article_id": "lifetime_articles_purchased",
                                              'price':"lieftime_amount_spent"})

In [24]:
historical_df = historical_df.drop(columns=['t_dat_max', 
                                            'customer_t_dat_max', 
                                            't_dat'])

## Data Mining Current Period Data

In [25]:
current_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
20808192,2020-01-01,0034b3dced3e565a43438bdfb5447e7321fea65388b398...,835247001,0.0339,2
20808193,2020-01-01,00410b91d62eefa76958fa5cac12f5daa7cfc0556e417d...,802930002,0.0678,2
20808194,2020-01-01,00410b91d62eefa76958fa5cac12f5daa7cfc0556e417d...,760084008,0.0254,2
20808195,2020-01-01,004b0fb384bcab2f8e1059dd5ca68c17580365ab95c05a...,804662002,0.0339,2
20808196,2020-01-01,004b0fb384bcab2f8e1059dd5ca68c17580365ab95c05a...,801554002,0.0169,2


In [26]:
current_df = current_df.groupby('customer_id').agg({'t_dat':max_min, 
                                                    'article_id':'count', 
                                                    'price':'sum'})

In [27]:
current_df = current_df.reset_index()

In [28]:
current_df['t_dat'] = pd.to_numeric(current_df['t_dat'].dt.days, downcast='integer')

In [29]:
current_df['quarter_average_amount_spent_per_article'] = current_df['price']/current_df['article_id']
current_df['quarter_average_days_between_purchase'] = current_df['price']/current_df['t_dat']

In [30]:
current_df['quarter_average_days_between_purchase'].replace(np.inf, 0, inplace = True)
current_df['quarter_average_amount_spent_per_article'].replace(np.inf, 0, inplace = True)

In [31]:
current_df = current_df.rename(columns={"article_id": "quarter_articles_purchased",
                                              'price':"quarter_amount_spent"})

In [32]:
current_df = current_df.drop(columns=['t_dat'])

### Saving to S3

In [33]:
wr.s3.to_csv(df = target_df,
             path = "s3://ads-508-group-6-processed/target_labels.csv",
             index = False)

{'paths': ['s3://ads-508-group-6-processed/target_labels.csv'],
 'partitions_values': {}}

In [34]:
wr.s3.to_csv(df = historical_df,
             path = "s3://ads-508-group-6-processed/historical_transactions.csv",
             index = False)

{'paths': ['s3://ads-508-group-6-processed/historical_transactions.csv'],
 'partitions_values': {}}

In [35]:
wr.s3.to_csv(df = current_df,
             path = "s3://ads-508-group-6-processed/quarter_transactions.csv",
             index = False)

{'paths': ['s3://ads-508-group-6-processed/quarter_transactions.csv'],
 'partitions_values': {}}

### Saving Locally

In [36]:
#target_df.to_csv("..\\data\\target_labels.csv", index = False)

In [37]:
#current_df.to_csv("..\\data\\current_transactions.csv", index = False)

In [38]:
#historical_df.to_csv("..\\data\\historical_transactions.csv", index = False)

### Shutting Down Kernel To Release Resources

In [39]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [40]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>